In [1]:
import pandas as pd

# ==========================================
# 1. 数据加载 (Data Loading)
# ==========================================
# 建议：如果你的文件在其他文件夹，请修改这里的路径
try:
    # 读取订单表 (核心时间表)
    df_orders = pd.read_csv('/Users/ethan/Desktop/Data Analyse/Brazilian E-Commerce Public Dataset by Olist/olist_orders_dataset.csv')
    
    # 读取商品明细表 (核心金钱表 - Price)
    df_items = pd.read_csv('/Users/ethan/Desktop/Data Analyse/Brazilian E-Commerce Public Dataset by Olist/olist_order_items_dataset.csv')
    
    # 读取用户表 (核心ID表 - Unique ID)
    df_customers = pd.read_csv('/Users/ethan/Desktop/Data Analyse/Brazilian E-Commerce Public Dataset by Olist/olist_customers_dataset.csv')
    
    print("✅ 数据加载成功！")
    print(f"Orders 数量: {df_orders.shape[0]}")
    print(f"Items 数量: {df_items.shape[0]}")
    print(f"Customers 数量: {df_customers.shape[0]}")
    
except FileNotFoundError:
    print("❌ 错误：找不到文件，请检查文件名或路径是否正确。")

# ==========================================
# 2. 数据清洗 (Data Cleaning)
# ==========================================

# ⚠️ 关键步骤：转换时间格式
# 原始数据是字符串，必须转成 datetime 才能做 RFM 计算
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])

# ⚠️ 业务逻辑过滤：只保留“已送达”的订单
# 只有真正完成的交易才算有效价值。取消(canceled)或未发货的通常不计入 RFM
df_orders_clean = df_orders[df_orders['order_status'] == 'delivered'].copy()

# ==========================================
# 3. 数据合并 (Data Merging) - 这里的顺序很重要！
# ==========================================

# 第一步：将“订单表”与“商品表”连接
# 目的：把“时间”和“金额(Price)”拼在一起
# 逻辑：一个订单可能有多个商品，所以行数会变多 (1对多)
merge_1 = pd.merge(
    df_orders_clean, 
    df_items, 
    on='order_id', 
    how='inner'  # 只保留有商品记录的订单
)

# 第二步：将“结果”与“用户表”连接
# 目的：找到订单背后的那个“唯一真实用户 (customer_unique_id)”
# 逻辑：一个订单对应一个临时 customer_id (1对1)
df_master = pd.merge(
    merge_1, 
    df_customers, 
    on='customer_id', 
    how='left'
)

# ==========================================
# 4. 最终检查 (Final Check)
# ==========================================

print("-" * 30)
print("🏆 宽表构建完成！(Master Table)")
print(f"最终行数: {df_master.shape[0]}")
print("-" * 30)

# 只保留我们需要分析的列，让表格清爽一点
required_cols = ['customer_unique_id', 'order_id', 'order_purchase_timestamp', 'price']
df_final = df_master[required_cols]

# 预览前 5 行
print(df_final.head())

✅ 数据加载成功！
Orders 数量: 99441
Items 数量: 112650
Customers 数量: 99441
------------------------------
🏆 宽表构建完成！(Master Table)
最终行数: 110197
------------------------------
                 customer_unique_id                          order_id  \
0  7c396fd4830fd04220f754e42b4e5bff  e481f51cbdc54678b7cc49136f2d6af7   
1  af07308b275d755c9edb36a90c618231  53cdb2fc8bc7dce0b6741e2150273451   
2  3a653a41f6f9fc3d2a113cf8398680e8  47770eb9100c2d0c44946d9cf07ec65d   
3  7c142cf63193a1473d2e66489a9ae977  949d5b44dbf5de918fe9c16f97b45f8a   
4  72632f0f9dd73dfee390c9b22eb56dd6  ad21c59c0840e6cb83a9ceb5573f8159   

  order_purchase_timestamp   price  
0      2017-10-02 10:56:33   29.99  
1      2018-07-24 20:41:37  118.70  
2      2018-08-08 08:38:49  159.90  
3      2017-11-18 19:28:06   45.00  
4      2018-02-13 21:18:39   19.90  


In [2]:
# ==========================================
# 1. 设定“分析基准日” (Snapshot Date)
# ==========================================
# 假设我们是在数据集最后一天过完后的第二天做分析
snapshot_date = df_final['order_purchase_timestamp'].max() + pd.Timedelta(days=1)
print(f"📅 分析基准日期设定为: {snapshot_date}")

# ==========================================
# 2. 计算 R, F, M (Aggregation)
# ==========================================
rfm = df_final.groupby('customer_unique_id').agg({
    'order_purchase_timestamp': lambda x: (snapshot_date - x.max()).days, # Recency
    'order_id': 'nunique',  # Frequency: 统计有多少个不重复的订单号
    'price': 'sum'          # Monetary: 消费总金额
})

# 重命名列名，方便后续引用
rfm.rename(columns={
    'order_purchase_timestamp': 'Recency',
    'order_id': 'Frequency',
    'price': 'Monetary'
}, inplace=True)

# ==========================================
# 3. 自动打分 (Scoring) - 运用我们刚才学的 Rank 技巧
# ==========================================
# 定义 1-5 分的标签
labels_good = [1, 2, 3, 4, 5]   # F和M：越大越好
labels_inverse = [5, 4, 3, 2, 1] # R：越小越好

# 使用 rank(method='first') 解决 Olist 数据集严重的“长尾分布”问题
# (因为 Olist 上绝大多数用户只买了一次，不强制排名的话无法切分)
rfm['R_Score'] = pd.qcut(rfm['Recency'].rank(method='first'), q=5, labels=labels_inverse)
rfm['F_Score'] = pd.qcut(rfm['Frequency'].rank(method='first'), q=5, labels=labels_good)
rfm['M_Score'] = pd.qcut(rfm['Monetary'].rank(method='first'), q=5, labels=labels_good)

# 拼接 RFM 标签 (例如 "555")
rfm['RFM_Segment'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)

# ==========================================
# 4. 成果展示
# ==========================================
print("✅ RFM 模型构建完成！")
print(rfm.head())

# 简单看下分布：谁是最好的客户？(555)
top_customers = rfm[rfm['RFM_Segment'] == '555']
print(f"\n🏆 超级核心用户 (555) 人数: {top_customers.shape[0]}")

📅 分析基准日期设定为: 2018-08-30 15:00:37
✅ RFM 模型构建完成！
                                  Recency  Frequency  Monetary R_Score  \
customer_unique_id                                                       
0000366f3b9a7992bf8c76cfdf3221e2      112          1    129.90       4   
0000b849f77a49e4a4ce2b2a4ca5be3f      115          1     18.90       4   
0000f46a3911fa3c0805444483337064      537          1     69.00       1   
0000f6ccb0745a6a4b88665a16c9f078      321          1     25.99       2   
0004aac84e0df4da2b147fca70cf8255      288          1    180.00       2   

                                 F_Score M_Score RFM_Segment  
customer_unique_id                                            
0000366f3b9a7992bf8c76cfdf3221e2       1       4         414  
0000b849f77a49e4a4ce2b2a4ca5be3f       1       1         411  
0000f46a3911fa3c0805444483337064       1       2         112  
0000f6ccb0745a6a4b88665a16c9f078       1       1         211  
0004aac84e0df4da2b147fca70cf8255       1       5        

In [3]:
# ==========================================
# 分析 1: 核心用户的营收贡献度 (The Power of "555")
# ==========================================
total_revenue = rfm['Monetary'].sum()
top_revenue = top_customers['Monetary'].sum()

print(f"💰 全站总营收: R$ {total_revenue:,.2f}")
print(f"💎 555用户贡献: R$ {top_revenue:,.2f}")
print(f"📈 仅占用户数 1% 的他们，贡献了营收的: {(top_revenue / total_revenue)*100:.2f}%")

# ==========================================
# 分析 2: 简单的用户分层画像 (Segment Profiling)
# ==========================================
# 为了让数据更直观，我们将所有用户简单分为 3 类进行对比
def simple_segment(row):
    # 只要 R_Score 是 1 (很久没买)，就算流失风险
    if str(row['R_Score']) == '1':
        return 'Hibernating (沉睡/流失)'
    # 如果 RFM 都是 4 或 5，就是冠军
    elif (int(row['R_Score']) >= 4) and (int(row['F_Score']) >= 4) and (int(row['M_Score']) >= 4):
        return 'Champions (核心)'
    else:
        return 'Others (普通)'

# 应用这个分类
rfm['Segment_Label'] = rfm.apply(simple_segment, axis=1)

# 统计每一组的平均 R, F, M 值
summary = rfm.groupby('Segment_Label').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
    'R_Score': 'count' # 统计人数
}).round(1)

# 重命名人数列
summary.rename(columns={'R_Score': 'Count'}, inplace=True)

print("\n📊 不同用户群体的画像对比：")
print(summary)

💰 全站总营收: R$ 13,221,498.11
💎 555用户贡献: R$ 391,071.23
📈 仅占用户数 1% 的他们，贡献了营收的: 2.96%

📊 不同用户群体的画像对比：
                     Recency  Frequency  Monetary  Count
Segment_Label                                           
Champions (核心)          90.9        1.2     274.2   6474
Hibernating (沉睡/流失)    473.2        1.0     140.5  18672
Others (普通)            187.5        1.0     129.4  68212


In [4]:
# 导出为 CSV 文件
# index=True 是为了把 customer_unique_id 也导出来
rfm.to_csv('olist_rfm_analysis_result.csv', index=True)

print("✅ 数据已导出为 'olist_rfm_analysis_result.csv'！")
print("🚀 下一步：打开 Tableau，连接这个文件。")

✅ 数据已导出为 'olist_rfm_analysis_result.csv'！
🚀 下一步：打开 Tableau，连接这个文件。
